In [1]:
%%HTML
<style type='text/css'>
    *{
        # background-color:#E3EDCD;
        # background-color:black;
        # color:white;
        
    }
    h1{
        color:#1976d2;
    }
    h2{
        color:#f57c00;
    }
    h3{
        color:#ba37ff;
    }
    h4{
        color:green;
    }
    table{
        border:1px solid black !important;
        border-collapse:collapse !important;
    }
    th{
        background-color:blueviolet !important;
        text-align:center;
        color:white;
    }
    th,td{
        border:0.1px solid black !important;
        transition:0.2s all liner;
        
    }
    td:hover{
        transform:scale(1.1);
        background-color:orange;
        color:blueviolet;
    }
    .raw{
        white-space:pre;
        color:green;
    }
    #imp{
        color:red;
    }
    #ct{
        text-align:center;
    }
    }
</style>

# 3.6 层级索引

当目前为止，我们接触的都是一维数据和二维数据，用 Pandas 的
Series 和 DataFrame 对象就可以存储。但我们也经常会遇到存储多维
数据的需求，数据索引超过一两个键。因此，Pandas 提供了 Panel(<a href="https://cn.bing.com/dict/search?q=panel&FORM=BDVSP2&qpvt=Panel" target='blank_'>仪表盘,控制板</a>) 和
Panel4D 对象解决三维数据与四维数据（详情请参见 3.7 节）。而在实
践中，更直观的形式是通过层级索引（hierarchical indexing，也被称为
多级索引，multi-indexing）配合多个有不同等级（level）的一级索引一
起使用，这样就可以将高维数组转换成类似一维 Series 和二维
DataFrame 对象的形式。  
在这一节中，我们将介绍创建 MultiIndex 对象的方法，多级索引数据
的取值、切片和统计值的计算，以及普通索引与层级索引的转换方法。
首先导入 Pandas 和 NumPy：  


In [2]:
import pandas as pd 
import numpy as np

## 3.6.1 多级索引Series

让我们看看如何用一维的 Series 对象表示二维数据——用一系列包含
特征与数值的数据点来简单演示。

01. 笨办法

假设你想要分析美国各州在两个不同年份的数据。如果你用前面介
绍的 Pandas 工具来处理，那么可能会用一个 Python 元组来表示索
引：

In [3]:
index = [('California', 2000), ('California', 2010),
            ('New York', 2000), ('New York', 2010),
            ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
                18976457, 19378102,
                20851820, 25145561]
pop = pd.Series(populations, index=index) #注意,这里的index只是一个普通的数组列表
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

通过元组构成的多级索引，你可以直接在 Series 上取值或用切片
查询数据：

In [4]:
pop[('California',2010):('Texas',2000)] #这里其实就是显式索引

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

但是这么做很不方便。假如你想要选择所有 2000 年的数据，那么
就得用一些比较复杂的（可能也比较慢的）清理方法了：

#### 简单理解下面的代码,pop.index获取的是行索引,我们要定位的元素是行索引元组中的年份,例如(California,2000),我们要定位的就是索引位置为1的2000

In [5]:
pop[[i for i in pop.index if i[1] == 2010]] #利用掩码

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

这么做虽然也能得到需要的结果，但是与 Pandas 令人爱不释手的
切片语法相比，这种方法确实不够简洁（在处理较大的数据时也不
够高效）。

02. <span id='imp'>好办法：Pandas多级索引(MultiIndex)</span>

好在 Pandas 提供了更好的解决方案。用元组表示索引其实是多级
索引的基础，Pandas 的 MultiIndex 类型提供了更丰富的操作方
法。我们可以用<span id='imp'>元组</span>创建一个多级索引，如下所示：

#### 简单理解,我们最开始用列表包纳了多个元组作为多级索引,这里的from_tuple很好理解,就是索引是由已经构建的元组得来的

In [6]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

```ipython
#按照书上的版本输出来的应该是这样的结果
Out[5]: MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])
```

你会发现 MultiIndex 里面有一个 levels 属性表示索引的等级
——这样做可以将州名和年份作为每个数据点的不同标签。

如果将前面创建的 pop 的索引<span id='imp'>重置（reindex）</span>为 MultiIndex，
就会看到层级索引：

#### 简单理解,就是重新编序,将pop的索引重新编序为新的索引(我们上面创建好的多级索引index)

In [7]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

其中前两列表示 Series 的多级索引值，第三列是数据。你会发现
有些行仿佛缺失了第一列数据——这其实是多级索引的表现形式，
每个空格与上面的索引相同。

现在可以直接用第二个索引获取 2010 年的全部数据，与 Pandas 的
切片查询用法一致：

#### 下面这行代码的语法含义是,切片获取数据,覆盖所有行,但是指定了列的显式索引必须是2010的

In [8]:
pop[:,2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

<span id='imp'>结果是单索引的数组</span>，正是我们需要的。与之前的元组索引相比，
多级索引的语法更简洁。（操作也更方便！）下面继续介绍层级索
引的取值操作方法。

03. 高维数据的多级索引

你可能已经注意到，我们其实完全可以用一个带行列索引的简单
DataFrame 代替前面的多级索引。其实 Pandas 已经实现了类似的
功能。<span id='imp'>unstack() 方法可以快速将一个多级索引的 Series 转化为
普通索引的 DataFrame：</span>

In [9]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


当然了，<span id='imp'>也有 stack() 方法实现相反的效果,即,将普通索引的DataFrame转化为多级索引的Series</span>

In [10]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

你可能会纠结于为什么要费时间研究层级索引。其实理由很简单：
如果我们可以用含多级索引的一维 Series 数据表示二维数据，那
么我们就可以用 Series 或 DataFrame 表示三维甚至更高维度的
数据。<span id='imp'>多级索引每增加一级，就表示数据增加一维</span>，利用这一特点
就可以轻松表示任意维度的数据了。假如要增加一列显示每一年各
州的人口统计指标（例如 18 岁以下的人口），那么对于这种带有
MultiIndex 的对象，增加一列就像 DataFrame 的操作一样简单：

#### 简单理解下面代码:回顾之前的DataFrame对象创建的方法,其中指定的key就是列名称,指定的数组列表就是这一列的数据

In [11]:
pop_df = pd.DataFrame({'total': pop, #pop是个带多级索引的Series对象,我们刚才创建过了
                        'under18': [9267089, 9284094,
                        4687374, 4318033,
                        5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

另外，所有在 3.4 节介绍过的通用函数和其他功能也同样适用于层
级索引。我们可以计算上面数据中 18 岁以下的人口占总人口的比
例：

In [12]:
f_u18 = pop_df['under18']/pop_df['total'] #这样得到的就是一个新的Series数组了
f_u18

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

#### 我们也可以使用unstack()来将多级索引的Series对象转换为DataFrame对象

In [13]:
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


同样，我们也可以快速浏览和操作高维数据

## 3.6.2 多级索引的创建方法

为 Series 或 DataFrame <span id='imp'>创建多级索引最直接的办法就是将 index 参
数设置为至少二维的索引数组</span>，如下所示：

In [14]:
df = pd.DataFrame(np.random.rand(4, 2), #数组的数据,0-1之间的随机数,是个4行2列的二维数组
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], #这样就创建了多级索引
                    columns=['data1', 'data2'])

In [15]:
df

data1     data2
a 1  0.074998  0.104470
  2  0.288507  0.375623
b 1  0.861621  0.650336
  2  0.646025  0.354390

MultiIndex 的创建工作将在后台完成。

同理，如果你把将元组作为键的字典传递给 Pandas，Pandas 也会默认转
换为 MultiIndex：

In [16]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data) #通过字典创建Series对象

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

但是有时候显式地创建 MultiIndex 也是很有用的，下面来介绍一些创
建方法。

01. 显式地创建多级索引

你可以用 pd.MultiIndex 中的类方法更加灵活地构建多级索引。
例如，就像前面介绍的，你可以通过一个有不同等级的若干简单数
组组成的列表来构建 MultiIndex：


#### 用数组来创建索引pd.MultiIndex.<span id='imp'>from_arrays(创建几级索引就写几维数组)</span>

In [17]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [18]:
pd.MultiIndex.from_arrays?

Signature:
pd.MultiIndex.from_arrays(
    arrays,
    sortorder: 'int | None' = None,
    names: 'Sequence[Hashable] | Hashable | lib.NoDefault' = <no_default>,
) -> 'MultiIndex'
Docstring:
Convert arrays to MultiIndex.

Parameters
----------
arrays : list / sequence of array-likes
    Each array-like gives one level's value for each data point.
    len(arrays) is the number of levels.
sortorder : int or None
    Level of sortedness (must be lexicographically sorted by that
    level).
names : list / sequence of str, optional
    Names for the levels in the index.

Returns
-------
MultiIndex

See Also
--------
MultiIndex.from_tuples : Convert list of tuples to MultiIndex.
MultiIndex.from_product : Make a MultiIndex from cartesian product
                          of iterables.
MultiIndex.from_frame : Make a MultiIndex from a DataFrame.

Examples
--------
>>> arrays = [[1, 1, 2, 2], ['red', 'blue', 'red', 'blue']]
>>> pd.MultiIndex.from_arrays(arrays, names=('number', 'color'))
MultiInd

也可以通过包含多个索引值的元组构成的列表创建 MultiIndex：

#### 上面用过的那种 pd.MultiIndex.<span id='imp'>from_tuple(里面用列表包住数组就行)</span>

In [19]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

还可以用两个索引的<span id='imp'>笛卡尔积（Cartesian product）创建
MultiIndex：</span>

In [20]:
pd.MultiIndex.from_product([['a','b'],[1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

#### 其实就是一级索引a和b分别都有俩2级索引1,2

更可以直接提供 levels（包含每个等级的索引值列表的列表）和
labels（包含每个索引值标签列表的列表）创建 MultiIndex：

<span id='imp'>其实写到这里,上面的labels已经过时了,被新来的codes替代了</span>

In [21]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [22]:
pd.MultiIndex?

Init signature:
pd.MultiIndex(
    levels=None,
    codes=None,
    sortorder=None,
    names=None,
    dtype=None,
    copy: 'bool' = False,
    name=None,
    verify_integrity: 'bool' = True,
) -> 'Self'
Docstring:     
A multi-level, or hierarchical, index object for pandas objects.

Parameters
----------
levels : sequence of arrays
    The unique labels for each level.
codes : sequence of arrays
    Integers for each level designating which label at each location.
sortorder : optional int
    Level of sortedness (must be lexicographically sorted by that
    level).
names : optional sequence of objects
    Names for each of the index levels. (name is accepted for compat).
copy : bool, default False
    Copy the meta-data.
verify_integrity : bool, default True
    Check that the levels/codes are consistent and valid.

Attributes
----------
names
levels
codes
nlevels
levshape
dtypes

Methods
-------
from_arrays
from_tuples
from_product
from_frame
set_levels
set_codes
to_frame
to_flat_

在创建 Series 或 DataFrame 时，可以将这些对象作为 index 参
数，或者通过 reindex 方法更新 Series 或 DataFrame 的索引。

02. 多级索引的等级名称

给 MultiIndex 的等级加上名称会为一些操作提供便利。你可以在
前面任何一个 MultiIndex 构造器中通过 names 参数设置等级名
称，也可以在创建之后通过索引的 names 属性来修改名称：

#### 简单理解,就是给有多级索引的列加上名字,最开始没有名字

In [23]:
pop.index.names = ['state','year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

在处理复杂的数据时，为等级设置名称是管理多个索引值的好办
法。

#### ↑就比如爬虫爬取一个省份不同区的楼盘,就可以将区名设计为多级索引中的第一级索引

03. 多级<span id='imp'>列索引</span>

#### 多级列索引在DataFrame中其实就是设置了列名

每个 DataFrame 的行与列都是对称的，也就是说既然有多级行索
引，那么同样可以有多级列索引。让我们通过一份医学报告的模拟
数据来演示：

In [24]:
# 多行列索引,注意,是多'行/列'索引,而不是多行'列'索引,即,多行索引和多列索引
# 注意这里用笛卡尔积里面的names其实就是上面.index.name那样为多级索引起名字
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                        names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                        names=['subject', 'type'])

In [25]:
#模拟数据

data = np.round(np.random.rand(4,6),1)
data[:,::2] *= 10
data+=37

In [26]:
# 创建DataFrame
health_data = pd.DataFrame(data,index=index,columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.0  37.2  39.0  37.1  46.0  37.3
     2      44.0  37.5  40.0  37.2  39.0  37.6
2014 1      47.0  37.6  41.0  37.4  45.0  37.7
     2      46.0  37.1  43.0  37.6  43.0  37.6

多级行列索引的创建非常简单。上面创建了一个简易的四维数据，
四个维度分别为被检查人的姓名、检查项目、检查年份和检查次
数。可以在列索引的第一级查询姓名，从而获取包含一个人（例如
Guido）全部检查信息的 DataFrame：

In [27]:
health_data['Guido']
#注意DataFrame默认是按照列名称进行索引的,如果不清楚就回看DataFrame的索引方式

type          HR  Temp
year visit            
2013 1      39.0  37.1
     2      40.0  37.2
2014 1      41.0  37.4
     2      43.0  37.6

如果想获取包含多种标签的数据，需要通过对多个维度（姓名、国
家、城市等标签）的多次查询才能实现，这时使用多级行列索引进
行查询会非常方便。


## 3.6.3 多级索引的取值与切片

对 MultiIndex 的取值和切片操作很直观，你可以直接把索引看成额外
增加的维度。我们先来介绍 Series 多级索引的取值与切片方法，再介
绍 DataFrame 的用法

01. Series多级索引

看看下面由各州历年人口数量创建的多级索引 Series：

In [28]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [29]:
pop.unstack() #只是回顾一下,这里不用写
#这里回顾一下unstack()将含有多级索引的series对象转换为带有普通索引的DataFrame对象

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


可以通过对多个级别索引值获取单个元素：

In [30]:
pop['California',2000] #获取加利福尼亚2000年的人口数据

np.int64(33871648)

MultiIndex 也支持局部取值（partial indexing），即只取索引的某
一个层级。假如只取最高级的索引，获得的结果是一个新的
Series，未被选中的低层索引值会被保留：

In [31]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

类似的还有局部切片，不过要求 MultiIndex 是按顺序排列的（就
像将在 3.6.4 节介绍的那样）：

In [32]:
pop.loc['California':'New Yourk']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

如果索引已经排序，那么可以用较低层级的索引取值，第一层级的
索引可以用空切片：

#### 下面一行代码的简单理解,就是行全选,列的话只要2000年的

In [33]:
pop[:,2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

其他取值与数据选择的方法（详情请参见 3.3 节）也都起作用。下
面的例子是通过布尔掩码选择数据：

In [34]:
pop[pop>22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

也可以用花哨的索引选择数据：

In [35]:
pop[['California','Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

02. DataFrame多级索引

DataFrame 多级索引的用法与 Series 类似。还用之前的体检报告
数据来演示：

In [36]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.0  37.2  39.0  37.1  46.0  37.3
     2      44.0  37.5  40.0  37.2  39.0  37.6
2014 1      47.0  37.6  41.0  37.4  45.0  37.7
     2      46.0  37.1  43.0  37.6  43.0  37.6

由于 DataFrame 的基本索引是列索引，因此 Series 中多级索引
的用法到了 DataFrame 中就应用在列上了。例如，可以通过简单
的操作获取 Guido 的心率数据：

In [37]:
health_data['Guido','HR']

year  visit
2013  1        39.0
      2        40.0
2014  1        41.0
      2        43.0
Name: (Guido, HR), dtype: float64

与单索引类似，在 3.3 节介绍的 loc(显式索引)、iloc(隐式索引) 和 ix 索引器都可以使
用，例如：

In [38]:
health_data.iloc[:2,:2] #这里用的是隐式索引,即取数据的第0行和第1行,第0列和第1列

subject      Bob      
type          HR  Temp
year visit            
2013 1      42.0  37.2
     2      44.0  37.5

虽然这些索引器将多维数据当作二维数据处理，<span id='imp'>但是在 loc 和
iloc 中可以传递多个层级的索引元组</span>，例如：

In [39]:
health_data.loc[:,('Bob','HR')]
#这里的意思就是行全取(index都保留),列只取Bob级下的HR,可以对比上面的图来看

year  visit
2013  1        42.0
      2        44.0
2014  1        47.0
      2        46.0
Name: (Bob, HR), dtype: float64

这种索引元组的用法不是很方便，如果在元组中使用切片还会导致
语法错误：


In [40]:
try:
    health_data.loc[(:,1),(:,'HR')]
except  SyntaxError as e:
    print(e);

SyntaxError: invalid syntax (2673756362.py, line 2)

虽然你可以用 Python 内置的 slice() 函数获取想要的切片，但是
还有一种更好的办法，就是使用 IndexSlice 对象。Pandas 专门用
它解决这类问题，例如：

<p class='raw'>简单理解下面的代码,创建一个索引对象  
然后利用显式索引,一级行索引都要,二级行索引要第一个次数的  
一级列索引都要,但是二级列索引只要‘HR’这一列的数据</p>

In [41]:
idx = pd.IndexSlice #我们先创建一个索引对象
health_data.loc[idx[:,1],idx[:,'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,42.0,39.0,46.0
2014,1,47.0,41.0,45.0


和带多级索引的 Series 和 DataFrame 进行数据交互的方法有很
多，但就像本书中的诸多工具一样，若想掌握它们，最好的办法就
是使用它们！


## 3.6.4 多级索引行列转换

使用多级索引的关键是掌握有效数据转换的方法。Pandas 提供了许多操
作，可以让数据在内容保持不变的同时，按照需要进行行列转换。之前
我们用一个简短的例子演示过 stack() 和 unstack() 的用法，但其实
还有许多合理控制层级行列索引的方法，让我们来一探究竟。

01. 有序的索引和无序的索引

在前面的内容里，我们曾经简单提过多级索引排序，这里需要详细
介绍一下。 <span id='imp'>如果 MultiIndex 不是有序的索引，那么大多数切片
操作都会失败。</span>让我们演示一下。  
首先创建一个不按字典顺序（lexographically）排列的多级索引
Series：  

In [42]:
#创建一个多级索引
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]]) 
# 数据准备,并指定行索引
data = pd.Series(np.random.rand(6),index=index)
# 给数据列命名
data.index.names = ['char','int']
data

char  int
a     1      0.568417
      2      0.562527
c     1      0.013964
      2      0.553747
b     1      0.961236
      2      0.295988
dtype: float64

如果想<span id='imp'>对索引使用局部切片，那么错误就会出现</span>：

In [43]:
try:
    data['a':'b'] #这里用的是显式索引
except BaseException as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


尽管从错误信息里面看不出具体的细节，但问题是出在
MultiIndex 无序排列上。<span id='imp'>局部切片和许多其他相似的操作都要求
MultiIndex 的各级索引是有序的（即按照字典顺序由 A 至 Z）。</span>
为此，Pandas 提供了许多便捷的操作完成排序，如 sort_index()
和 sortlevel() 方法。我们用最简单的 sort_index() 方法来演示：


In [44]:
data = data.sort_index()
data

char  int
a     1      0.568417
      2      0.562527
b     1      0.961236
      2      0.295988
c     1      0.013964
      2      0.553747
dtype: float64

索引排序之后，局部切片就可以正常使用了：

In [45]:
data['a':'b']

char  int
a     1      0.568417
      2      0.562527
b     1      0.961236
      2      0.295988
dtype: float64

02. 索引stack与unstack

前文曾提过，我们可以将一个多级索引数据集转换成简单的二维形
式，<span id='imp'>可以通过 level 参数设置转换的索引层级：</span>

#### 我们先来看一下unstack的一些参数

In [46]:
pop.unstack?

Signature:
pop.unstack(
    level: 'IndexLabel' = -1,
    fill_value: 'Hashable | None' = None,
    sort: 'bool' = True,
) -> 'DataFrame'
Docstring:
Unstack, also known as pivot, Series with MultiIndex to produce DataFrame.

Parameters
----------
level : int, str, or list of these, default last level
    Level(s) to unstack, can pass level name.
fill_value : scalar value, default None
    Value to use when replacing NaN values.
sort : bool, default True
    Sort the level(s) in the resulting MultiIndex columns.

Returns
-------
DataFrame
    Unstacked Series.

Notes
-----
Reference :ref:`the user guide <reshaping.stacking>` for more examples.

Examples
--------
>>> s = pd.Series([1, 2, 3, 4],
...               index=pd.MultiIndex.from_product([['one', 'two'],
...                                                 ['a', 'b']]))
>>> s
one  a    1
     b    2
two  a    3
     b    4
dtype: int64

>>> s.unstack(level=-1)
     a  b
one  1  2
two  3  4

>>> s.unstack(level=0)
   one  two
a    1

#### 注意观察上面的实例代码,还是很生动形象的

先来看看不指定参数的原始数据的排布样式

In [47]:
pop.unstack()

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [48]:
pop.unstack(level=1) #可以看到和上面的结果是一样的,默认就是level=1

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [49]:
pop.unstack(level=0) #接下来尝试一下level=0

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


#### 可以发现与原来的相比,level=0使得轴线发生变换,行变成了列,列变成了行

In [50]:
pop.unstack(level=-1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


unstack() 是 stack() 的逆操作，同时使用这两种方法让数据保
持不变：

#### 我们先来看看原始的pop

In [51]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### 然后我们将其unstack把它变成一个二维的DataFrame

In [52]:
pop.unstack()

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


#### 接下来我们验证一下其逆操作会不会把unstack()后的数据变回原始的DataFrame对象

In [53]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### ↑ 发现成功了,变回原来的pop的带有多级索引的Series对象

03. 索引的设置与重置

层级数据维度转换的另一种方法是行列标签转换，可以通过
reset_index 方法实现。如果在上面的人口数据 Series 中使用该
方法，则会<span id='imp'>生成一个列标签中包含之前行索引标签 state 和 year 的
DataFrame。</span>也可以用数据的 name 属性为列设置名称：

In [54]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


在解决实际问题的时候，如果能将类似这样的原始输入数据的列直
接转换成 MultiIndex，通常将大有裨益。其实可以通过
DataFrame 的 set_index 方法实现，返回结果就会是一个带多级
索引的 DataFrame：

<p class='raw'>简单理解下面的一行代码,其实就是设置state和year为层级索引
</p>

In [55]:
pop_flat.set_index(['state','year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

在实践中，我发现用这种重建索引的方法处理数据集非常好用

## 3.6.5 多级索引的数据累计方法

前面我们已经介绍过一些 Pandas 自带的数据累计方法，比如
mean()、sum() 和 max()。而对于层级索引数据，可以设置参数
level 实现对数据子集的累计操作。

再一次以体检数据为例

In [56]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.0  37.2  39.0  37.1  46.0  37.3
     2      44.0  37.5  40.0  37.2  39.0  37.6
2014 1      47.0  37.6  41.0  37.4  45.0  37.7
     2      46.0  37.1  43.0  37.6  43.0  37.6

如果你需要计算每一年各项指标的平均值，那么可以将参数 level 设
置为索引 year：

In [57]:
data_mean = health_data.mean(level='year')
data_mean

TypeError: mean() got an unexpected keyword argument 'level'